# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import sys
from io import StringIO

# Evaluate Llama-2 Trained to Generate Python Functions

In [2]:
file_path = 'questions_5shot.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("\n\n### Response: ")
    text = " ".join(sections)
    sections = text.split("### Response: ")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    question_count=0
    for section in sections[0:100]:
        if "\nBelow" in section:
            section = section.split("\nBelow")[0]
        try:
            question = section.split("Question:")[1].strip()
            question_count+=1
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        data.append({"question": question, "solution": solution_text, 'answer': answer})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            data.append({"question": question, "solution": solution_text, 'answer': answer})
                        except:
                            pass
                except:
                    pass
        except:
            pass

In [3]:
df = pd.DataFrame.from_dict(data)
df.to_csv("llama_v1.csv")
result1 = "Percentage of Questions with Executable Code: " + str((len(df)/100)*100)
result1

'Percentage of Questions with Executable Code: 28.999999999999996'

In [4]:
df

,question,solution,answer
0,A coder has 1000 followers on Twitter. He gain...,def solution():\n #The coder has 1000 follo...,1500.0
1,A 20-year-old man is 66 inches tall. His 10-y...,def solution():\n #The 20-year-old is 66 in...,44.0
2,A bakery has 100 muffins. The bakery sells 20...,def solution():\n #The bakery has 100 muffi...,50.0
3,A baker is making a 3-layer cake. The first la...,def solution():\n #The first layer is 10 in...,628.0
4,A pizzeria is making pizzas for a party. 12 of...,def solution():\n #The pizzeria is making 2...,4.0
5,A 10-pound sack of potatoes costs $5. A 5-poun...,def solution():\n #A 10-pound sack of potat...,7.5
6,The soccer team has 24 players. 12 of the play...,def solution():\n #The team has 24 players\...,6.0
7,A 10-pound sack of apples costs $22. A 20-poun...,def solution():\n #10 pounds of apples cost...,24.2
8,A bakery sells 1200 cookies a day. They bake 1...,def solution():\n #The bakery sells 1200 co...,800.0
9,A 10-pound bag of apples has 40 apples. How ma...,def solution():\n #10 pounds of apples has ...,3.0


In [5]:
df = pd.read_csv("llama_v1_annotated.csv")
appropriate = "Appropriateness: " + str((len(df[df['appropriateness']==1])/len(df))*100)
solvable = "Solvable: " + str((len(df[df['solvability']==1])/len(df))*100)
accurate = "Accurate: " + str((len(df[df['solution accuracy']==1])/len(df))*100)
appropriate

'Appropriateness: 80.0'

In [6]:
solvable

'Solvable: 88.0'

In [7]:
accurate

'Accurate: 80.0'

# Evaluate Llama-2 Trained to Generate Python Code

In [8]:
file_path = 'llama_QA_generate_few_shot.txt'
data = []
with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Prompting Approach: few shot. Generated Text: ### Response: ")
    del sections[0]
    total_questions = len(sections)
    for section in sections:
        try:
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            count+=1
            initial_vars = dict(locals().items())
            old_stdout = sys.stdout
            redirected_output = sys.stdout = StringIO()
            try:
                if 'print(' in solution:
                    exec(solution, globals(), locals())
                    sys.stdout = old_stdout

                    value = redirected_output.getvalue().strip()
                    data.append({"question": question, "solution": solution, 'answer': value})

                    # Create a dictionary to store the variables after execution
                    vars_after_exec = dict(locals().items())

                    # Find the new variables created by exec()
                    new_variables = {var_name: var_value for var_name, var_value in vars_after_exec.items() if var_name not in initial_vars}

                    # Delete the new local variables
                    for var_name in new_variables:
                        del locals()[var_name]

            except Exception as e:
                pass
        except:
            pass

In [9]:
df = pd.DataFrame.from_dict(data)
df.to_csv("llama_QA.csv")
result1 = "Percentage of Questions with Executable Code: " + str((len(df)/total_questions)*100)
print(result1)
print(len(df), total_questions)

Percentage of Questions with Executable Code: 0.0
0 53


In [10]:
#Conduct significance test to see whether Python function prompts are better than python code prompts
import numpy as np
from scipy import stats
function = np.concatenate([np.ones(29), np.zeros(71)])
code = np.concatenate([np.ones(10), np.zeros(43)])

# Perform an independent two-sample t-test
t_statistic, p_value = stats.ttest_ind(function, code)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


T-statistic: 1.3677725812124728
P-value: 0.1734154443919006


In [11]:
.1887*53

10.001100000000001

In [12]:
df

""


In [13]:
df = pd.read_csv("llama_QA_annotated.csv")
appropriate = "Appropriateness: " + str((len(df[df['appropriateness']==1])/len(df))*100)
solvable = "Solvable: " + str((len(df[df['solvability']==1])/len(df))*100)
accurate = "Accurate: " + str((len(df[df['solution accuracy']==1])/len(df))*100)
appropriate

'Appropriateness: 60.0'

In [14]:
solvable

'Solvable: 60.0'

In [15]:
accurate

'Accurate: 80.0'

# Evaluate Llama trained to generate code prompted to generate python functions instead of traditional code

In [16]:
file_path = 'llama_QA_generate_function.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("\n\n### Response: ")
    text = " ".join(sections)
    sections = text.split("### Response: ")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    question_count=0
    for section in sections:
        if "\nBelow" in section:
            section = section.split("\nBelow")[0]
        try:
            question = section.split("Question:")[1].strip()
            question_count+=1
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        data.append({"question": question, "solution": solution_text, 'answer': answer})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            data.append({"question": question, "solution": solution_text, 'answer': answer})
                        except:
                            pass
                except:
                    pass
        except:
            pass

In [17]:
df = pd.DataFrame.from_dict(data)
df.to_csv("llama_QA_function.csv")

In [18]:
result = "Percentage of Questions with Executable Code: " + str((len(df)/total_questions)*100)
result

'Percentage of Questions with Executable Code: 21.052631578947366'

In [19]:
df

,question,solution,answer
0,A hamburger at a fast-food restaurant costs $4...,def solution():\n #Cost of a hamburger\n ...,6.000000e+00
1,40000 people live in a city. 20% of the popula...,def solution():\n #Total population of the ...,8.000000e+03
2,"10,000,000 is the product of two 6-digit numbe...","def solution():\n #10,000,000 is the produc...",1.000000e+03
3,150 is a multiple of 15. 15 is a factor of 150...,def solution():\n #200 is the number we are...,2.000000e+02
4,1000 people are at a concert. 200 of them are ...,def solution():\n #1000 people are at a con...,4.000000e+01
5,1200 people were at a rally. 200 of them left....,def solution():\n #There were 1200 people a...,3.333333e+02
6,A soccer team has 22 players. 11 of the player...,def solution():\n #Total number of players\...,0.000000e+00
7,The 24-hour gym is open 24 hours a day. 1200 p...,def solution():\n #Total number of people w...,-1.000000e+02
8,2355 is the 14th largest 4-digit number. \nWha...,def solution():\n #4-digit numbers start at...,1.900000e+01
9,12000 people live in a small town. 10000 of th...,def solution():\n #12000 people live in the...,2.000000e+03


In [20]:
df = pd.read_csv("llama_QA_function_annotated.csv")
appropriate = "Appropriateness: " + str((len(df[df['appropriateness']==1])/len(df))*100)
solvable = "Solvable: " + str((len(df[df['solvability']==1])/len(df))*100)
accurate = "Accurate: " + str((len(df[df['solution accuracy']==1])/len(df))*100)
appropriate

'Appropriateness: 61.53846153846154'

In [21]:
solvable

'Solvable: 76.92307692307693'

In [22]:
accurate

'Accurate: 61.53846153846154'

# Evaluate Mammoth Trained to Generate Questions

In [23]:
file_path = 'mammoth_question_generate_v2.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Prompting Approach: few shot. Generated Text: ### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            question = section.split("Question:")[1].strip()
            data.append({"question": question})
        except:
            pass
            #pass if there is no question

In [24]:
df = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions: " + str((len(df)/total_questions)*100)
result1

'Percentage of Questions: 25.0'

In [25]:
df.to_csv("mammoth_question.csv")
df

,question
0,What is the value of $x$ for which $(x+2)^2 = ...
1,What is the sum of all two-digit positive inte...
2,What's the total number of colors on RGB color...
3,What is the sum of all two-digit positive inte...
4,"If you buy a dozen oranges for $6, and 3-for-$..."
5,What is the greatest three digit number that i...
6,"If $f(n) = n^2 + n + 17$, what is $f(11)$?"
7,There are 325 students in a class. 20% of them...
8,"If $x$ is a positive multiple of $3$, and $x^2..."
9,How many 30-minute shows can Lexy watch in 2 h...


In [26]:
df = pd.read_csv("mammoth_question_annotated.csv")
appropriate = "Appropriateness: " + str((len(df[df['appropriateness']==1])/len(df))*100)
solvable = "Solvable: " + str((len(df[df['solvability']==1])/len(df))*100)
#accurate = "Accurate: " + str((len(df[df['solution accuracy']==1])/len(df))*100)
appropriate

'Appropriateness: 40.0'

In [27]:
solvable

'Solvable: 96.0'

# Topic Analysis:

## Mammoth Question Model

In [28]:
file_path = 'mammoth_question_topic_generate.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a grade school math word problem about Superman.\n\n### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    counter = 0
    question_count = 0
    for section in sections:
        if counter<len(sections)-1:
            section = section.split("Prompting Approach: few shot. Generated Text: ### Response: Question:")[0]
            try:
                question = section.split("Question:")[1].strip()
                question_count+=1
                if "Superman" in question:
                    data.append({"question": question})
            except:
                pass
                #pass if there is no question
        counter+=1
df = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions: " + str((len(df)/total_questions)*100)
result2 = "Percentage of Questions that Contain Topic: " + str((len(df)/question_count)*100)
result1

'Percentage of Questions: 19.0'

In [29]:
result2

'Percentage of Questions that Contain Topic: 100.0'

In [30]:
df.to_csv("mammoth_question_topic.csv")
df

,question
0,If Superman can lift 50 times more than a huma...
1,"If Superman can lift 5000 pounds of weight, an..."
2,If Superman can lift 5000 pounds above his hea...
3,If Superman can lift 1000 times more than 20 t...
4,Superman can lift 1000 times his own body weig...
5,Superman can lift 100 pounds more than three t...
6,How many hours will it take for Superman to sa...
7,How many more days than Batman can Superman la...
8,Superman is flying across the country at 3 tim...
9,Superman's cape weighs 10 pounds when dry. Wh...


In [31]:
df = pd.read_csv("mammoth_question_topic_annotated.csv")
appropriate = "Appropriateness: " + str((len(df[df['appropriateness']==1])/len(df))*100)
solvable = "Solvable: " + str((len(df[df['solvability']==1])/len(df))*100)
#accurate = "Accurate: " + str((len(df[df['solution accuracy']==1])/len(df))*100)
appropriate

'Appropriateness: 94.73684210526315'

In [32]:
solvable

'Solvable: 52.63157894736842'

## Llama QA Model

In [33]:
file_path = 'llama_QA_generate_topic.txt'
def evaluate(file_path):
    data = []
    with open(file_path, 'r') as file:
        text = file.read()
        sections = text.split("Prompting Approach: few shot. Generated Text: ### Response:")
        del sections[0]
        total_questions = len(sections)
        text = " ".join(sections)
        sections = text.split("Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a grade school math word problem about Superman and Python program to solve the word problem.\n\n### Response:")
        del sections[0]
        print(total_questions)
        question_count=0
        for section in sections:
            try:
                section = section.split("Prompting Approach: few shot. Generated Text: ### Response: Question:")[0]
                question = section.split("Question:")[1].strip()
                question_count+=1
                question = question.split("Solution:")[0].strip()
                solution = section.split("Solution:")[1].strip()
                if "\nBel" in solution:
                    solution = solution.split("\nBel")[0]
                initial_vars = dict(locals().items())
                old_stdout = sys.stdout
                redirected_output = sys.stdout = StringIO()
                try:
                    if 'print(' in solution:
                        exec(solution, globals(), locals())
                        sys.stdout = old_stdout

                        value = redirected_output.getvalue().strip()
                        data.append({"question": question, "solution": solution, 'answer': value})

                        # Create a dictionary to store the variables after execution
                        vars_after_exec = dict(locals().items())

                        # Find the new variables created by exec()
                        new_variables = {var_name: var_value for var_name, var_value in vars_after_exec.items() if var_name not in initial_vars}

                        # Delete the new local variables
                        for var_name in new_variables:
                            del locals()[var_name]

                except Exception as e:
                    pass
            except:
                pass
    print("Percentage of Questions with Executable Code: " + str((len(data)/total_questions)*100))
    print("Percentage of Executable Code that Contain Topic: " + str((len(data)/question_count)*100))
    return data, total_questions, question_count
output = evaluate(file_path)

52


In [34]:
df = pd.DataFrame.from_dict(output[0])
df.to_csv("llama_QA_topic.csv")
df
#Cannot figure out why Dino is returned here

,question,solution,answer
0,"Superman can fly at a top speed of 4,000 miles...",# define the top speed and the percentage of t...,12000.0
1,Superman's cape is 50% longer than his height....,# define Superman's height in feet\nheight_in_...,108.0
2,Superman can fly at a top speed of 4000 miles ...,# define the variables\nsuperman_speed = 4000\...,0.25
3,Superman can bench press 1000 pounds. He is o...,# define the variables\nsuperman_strength = 10...,10.0
4,"Superman is 6'5"" tall. He is 1000 miles from ...",# Define the variables\nmonthly_income = 6000\...,2280.0
5,Superman can fly at a top speed of 2500 miles ...,# define the distance and speed\ndistance = 28...,2.24
6,Superman's cape is 50% longer than his body. ...,# define the height of Superman in feet\nheigh...,115.20000000000002
7,Superman can fly at a top speed of 2500 miles ...,# top speed of Superman\ntop_speed = 2500\n\n#...,2.4
8,Superman can fly at a speed of 2000 miles per ...,# define the speed and distance\nspeed = 2000 ...,10.0
9,Superman can fly at a top speed of 2500 miles ...,# define the speeds\nsuperman_speed = 2500\nwo...,-3.2


In [35]:
result1 = "Percentage of Questions with Executable Code: " + str((len(df)/output[1])*100)
result1

'Percentage of Questions with Executable Code: 28.846153846153843'

In [36]:
result2 = "Percentage of Executable Code that Contains Topic: " + str((len(df[df['question'].str.contains('Superman')])/len(df))*100)
result2

'Percentage of Executable Code that Contains Topic: 93.33333333333333'

In [37]:
df = pd.read_csv("llama_QA_topic_annotated.csv")
appropriate = "Appropriateness: " + str((len(df[df['appropriateness']==1])/len(df))*100)
solvable = "Solvable: " + str((len(df[df['solvability']==1])/len(df))*100)
accurate = "Accurate: " + str((len(df[df['solution accuracy']==1])/len(df))*100)
appropriate

'Appropriateness: 100.0'

In [38]:
solvable

'Solvable: 83.33333333333334'

In [39]:
accurate

'Accurate: 91.66666666666666'

# Llama QA Model Prompted to Generate Python Functions

In [40]:
file_path = 'llama_QA_function_topic.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("\n\n### Response: ")
    text = " ".join(sections)
    sections = text.split("### Response: ")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    question_count=0
    for section in sections:
        if "\nBelow" in section:
            section = section.split("\nBelow")[0]
        try:
            question = section.split("Question:")[1].strip()
            question_count+=1
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        data.append({"question": question, "solution": solution_text, 'answer': answer})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            data.append({"question": question, "solution": solution_text, 'answer': answer})
                        except:
                            pass
                except:
                    pass
        except:
            pass

In [41]:
df = pd.DataFrame.from_dict(data)
df.to_csv("llama_QA_function_topic.csv")
df

,question,solution,answer
0,Superman can leap 1000 feet in a single bound....,def solution():\n #Superman can leap 1000 f...,50000.000000
1,Superman can leap 2000 feet in a single bound....,def solution():\n #Superman can leap 2000 f...,189.393939
2,Superman can leap 1000 feet in a single bound....,def solution():\n #Superman leaps 1000 feet...,1.893939
3,"Superman can leap 1,000 feet in a single bound...","def solution():\n #Superman can leap 1,000 ...",3000.000000
4,Superman can run at a speed of 250 miles per h...,def solution():\n #Superman's speed\n su...,0.000000
...,...,...,...
74,Superman can leap 6000 feet in a single bound....,def solution():\n #Superman can leap 6000 f...,1.666667
75,Superman can leap a 20-story building in a sin...,def solution():\n #Superman can leap a 20-s...,240.000000
76,Superman can leap 1000 feet in a single bound....,def solution():\n #Superman leaped 500 feet...,0.000000
77,"Superman can fly at a speed of 3,000 miles per...","def solution():\n #Superman is 1,500 miles ...",-1500.000000


In [42]:
result1 = "Percentage of Questions with Executable Code: " + str((len(df)/total_questions)*100)
result1

'Percentage of Questions with Executable Code: 80.61224489795919'

In [43]:
result2 = "Percentage of Executable Code that Contains Topic: " + str((len(df[df['question'].str.contains('Superman')])/len(df))*100)
result2

'Percentage of Executable Code that Contains Topic: 97.46835443037975'

# Original Llama Model Trained to Generate Python Functions

In [44]:
file_path = 'llama_v1_topic.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("\n\n### Response: ")
    text = " ".join(sections)
    sections = text.split("### Response: ")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    question_count=0
    for section in sections:
        if "\nBelow" in section:
            section = section.split("\nBelow")[0]
        try:
            question = section.split("Question:")[1].strip()
            question_count+=1
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        data.append({"question": question, "solution": solution_text, 'answer': answer})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            data.append({"question": question, "solution": solution_text, 'answer': answer})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df)/total_questions)*100)
result1

'Percentage of Questions with Executable Code: 37.34939759036144'

In [45]:
result2 = "Percentage of Executable Code that Contains Topic: " + str((len(df[df['question'].str.contains('Superman')])/len(df))*100)
result2

'Percentage of Executable Code that Contains Topic: 100.0'

In [46]:
df

,question,solution,answer
0,Superman can fly at 250 miles per hour. He is ...,def solution():\n #Superman is 1500 miles a...,6.000000e+00
1,Superman can fly at a top speed of 250 miles p...,def solution():\n #Superman's top speed is ...,8.000000e+00
2,Superman can fly at a speed of 2500 miles per ...,def solution():\n #Superman's speed is 2500...,9.800000e-01
3,Superman can fly at 2500 miles per hour. He le...,def solution():\n #Superman left Metropolis...,3.120000e+00
4,Superman can run at 250 miles per hour. He' a ...,def solution():\n #Superman can run at 250 ...,8.000000e+00
5,Superman flies from the Earth to the Moon in 3...,def solution():\n #Superman flies from the ...,2.804800e+07
6,Superman can fly at 250 miles per hour. He's 1...,def solution():\n #Superman is 1200 miles a...,4.800000e+00
7,Superman can fly at the speed of light. He lef...,def solution():\n #Superman left Metropolis...,2.000000e+00
8,Superman can fly at 250 miles per hour. He's i...,def solution():\n #Superman flies at 250 mp...,6.400000e+00
9,Superman can fly at a speed of 500 miles per h...,def solution():\n #Superman's flying speed ...,1.200000e+03


In [47]:
df.to_csv("llama_v1_topic.csv")

## Llama V1 Prompted to Generate Python Functions (using word Python Function)

In [48]:
file_path = 'llama_v1_topic_v2.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        data.append({"question": question, "solution": solution_text, 'answer': answer})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            data.append({"question": question, "solution": solution_text, 'answer': answer})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        data.append({"question": question, "solution": solution_text, 'answer': answer})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            data.append({"question": question, "solution": solution_text, 'answer': answer})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df)/total_questions)*100)
result1

'Percentage of Questions with Executable Code: 43.43434343434344'

In [49]:
result2 = "Percentage of Executable Code that Contains Topic: " + str((len(df[df['question'].str.contains('Superman')])/len(df))*100)
result2

'Percentage of Executable Code that Contains Topic: 100.0'

In [50]:
df

,question,solution,answer
0,Superman can fly at 250 miles per hour. He lef...,def solution():\n #Superman can fly at 250 ...,3.0
1,Superman can fly at 250 miles per hour. He is ...,def solution():\n #Superman flies at 250 mi...,8.0
2,Superman can fly at 2500 miles per hour. He is...,def solution():\n #Superman can fly 2500 mi...,0.8
3,Superman can fly at a speed of 250 miles per h...,def solution():\n #Superman's speed is 250 ...,5.0
4,Superman can run at 250 miles per hour. The Fl...,def solution():\n #Superman can run at 250 ...,450
5,Superman can fly at a speed of 250 miles per h...,def solution():\n #Superman's speed is 250 ...,6.0
6,Superman can fly at a speed of 250 miles per h...,def solution():\n #Superman's speed is 250 ...,8.0
7,Superman can fly at a speed of 250 miles per h...,def solution():\n #Superman can fly at 250 ...,10.4
8,Superman can fly at 2500 miles per hour. He ne...,def solution():\n #Superman can fly 2500 mi...,1.0
9,Superman can fly at a speed of 250 miles per h...,def solution():\n #Superman can fly 250 mil...,6.0


In [51]:
df.to_csv("llama_v1_topic_v2.csv")

## Llama QA Prompted to Generate Python Functions but with additional parsing

In [52]:
file_path = 'llama_QA_function_topic_v2.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "\nBel" in solution:
                solution = solution.split("\nBel")
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        data.append({"question": question, "solution": solution_text, 'answer': answer})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            data.append({"question": question, "solution": solution_text, 'answer': answer})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        data.append({"question": question, "solution": solution_text, 'answer': answer})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            data.append({"question": question, "solution": solution_text, 'answer': answer})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df)/total_questions)*100)
result1

'Percentage of Questions with Executable Code: 86.73469387755102'

In [53]:
result2 = "Percentage of Executable Code that Contains Topic: " + str((len(df[df['question'].str.contains('Superman')])/len(df))*100)
result2

'Percentage of Executable Code that Contains Topic: 98.82352941176471'

In [54]:
df

,question,solution,answer
0,Superman can leap 1000 feet in a single bound....,def solution():\n #Superman can leap 1000 f...,30000
1,Superman is 6.25 feet tall. He is 12.5 times ...,def solution():\n #Superman is 6.25 feet ta...,150.0
2,Superman can run at a speed of 3000 miles per ...,def solution():\n # Superman's speed in mil...,4.0
3,Superman can fly at 4000 miles per hour. He is...,def solution():\n #Superman can fly at 4000...,0.5
4,"Superman is 6'3"" tall. He is 20% taller than ...","def solution():\n #Superman is 6'3""\n su...",5.25
...,...,...,...
80,Superman can punch 100 times per second. He p...,def solution():\n #Superman punches 100 tim...,30000
81,"Superman is 6'3"" and wears a cape that is 5.5'...",def solution():\n #Superman's height\n s...,-9.6
82,Superman can leap 1000 feet in a single bound....,def solution():\n #Superman can leap 1000 f...,20000
83,Superman can leap 2000 feet in a single bound....,def solution():\n #Superman can leap 2000 f...,0.5


In [55]:
df.to_csv("llama_QA_function_topic_v2.csv")